<a href="https://colab.research.google.com/github/angrycircuit/mie1624-group1-project/blob/jacky-wip/university_course_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Waterloo University Course Lists

In [2]:
import numpy as np
import pandas as pd
import urllib
from bs4 import BeautifulSoup
import requests

import matplotlib.pyplot as plt
import numpy as np
## For Preparing Features to Model
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer


try:
    from wordcloud import WordCloud
except:
    !pip install wordcloud
    from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
url="https://uwaterloo.ca/graduate-studies-academic-calendar/mathematics/data-science-and-artificial-intelligence/master-data-science-and-artificial-intelligence-mdsai-co-operative-program-direct-entry#coursework"
resp = requests.get(url)
soup = BeautifulSoup(resp.content)

In [4]:
t=[]

for text in soup.find_all(name="div", attrs={"class":"field-content field-item"}):
  for li in text.find_all(name="li"):  
    t.append(li.text)
  

In [6]:
waterloo_df = pd.DataFrame(t, columns=['course_list'])
waterloo_df['course_list'].replace(inplace=True,regex=True,to_replace="\n",value="")
waterloo_df['course_list'] = waterloo_df['course_list'].str.lower()
waterloo_df = waterloo_df.loc[waterloo_df['course_list'].str.startswith('co ') | waterloo_df['course_list'].str.startswith('cs ') | waterloo_df['course_list'].str.startswith('stat ')]
waterloo_df.drop_duplicates(inplace=True)

waterloo_df['department'] = waterloo_df['course_list'].str.split().str[0]
waterloo_df['code'] = waterloo_df['course_list'].str.split().str[1]

waterloo_df

,course_list,department,code
3,cs 600 fundamentals of computer science for da...,cs,600
4,stat 845 statistical concepts for data science...,stat,845
7,stat 847 exploratory data analysis,stat,847
9,cs 651 data-intensive distributed computing (d...,cs,651
10,cs 631 data-intensive distributed analytics (d...,cs,631
12,stat 841 statistical learning - classification,stat,841
13,stat 842 data visualization,stat,842
14,stat 844 statistical learning - function estim...,stat,844
16,cs 638 principles of data management and use,cs,638
17,cs 648 database systems implementation,cs,648


In [7]:
url_co = "https://uwaterloo.ca/graduate-studies-academic-calendar/graduate-course/subject/CO"
url_cs = "https://uwaterloo.ca/graduate-studies-academic-calendar/graduate-course/subject/CS"
url_stat = "https://uwaterloo.ca/graduate-studies-academic-calendar/graduate-course/subject/STAT"

htm_co = urllib.request.urlopen(url_co).read()
soup_co = BeautifulSoup(htm_co, "html.parser")

htm_cs = urllib.request.urlopen(url_cs).read()
soup_cs = BeautifulSoup(htm_cs, "html.parser")

htm_stat = urllib.request.urlopen(url_stat).read()
soup_stat = BeautifulSoup(htm_stat, "html.parser")

In [12]:
course_list = pd.DataFrame(columns=['title', 'code', 'desciption', 'university'])

In [14]:
university = "waterloo"

for course_soup in soup_co.find_all('div', attrs = {'class': 'field-content'}):
    department = 'co'
    code = course_soup.find('a').get('name')
    title = course_soup.find('h2').find('a').text
    desciption = course_soup.find('div', attrs = {'class': 'course_des'}).text

    course_list = course_list.append({"title": title, "desciption": desciption, "code": department + code, "university": university}, ignore_index=True)
for course_soup in soup_cs.find_all('div', attrs = {'class': 'field-content'}):
    department = 'cs'
    code = course_soup.find('a').get('name')
    title = course_soup.find('h2').find('a').text
    desciption = course_soup.find('div', attrs = {'class': 'course_des'}).text

    course_list = course_list.append({"title": title, "desciption": desciption, "code": department + code, "university": university}, ignore_index=True)

for course_soup in soup_stat.find_all('div', attrs = {'class': 'field-content'}):
    department = 'stat'
    code = course_soup.find('a').get('name')
    title = course_soup.find('h2').find('a').text
    desciption = course_soup.find('div', attrs = {'class': 'course_des'}).text

    course_list = course_list.append({"title": title, "desciption": desciption, "code": department + " " + code, "university": university}, ignore_index=True)

course_list

,title,code,desciption,university
0,Fundamentals of Enumerative Combinatorics,co601,"Elementary counting, bijections. Sets, weights...",waterloo
1,Fundamentals of Optimization,co602,"Linear optimization: Farkas' Lemma, duality, S...",waterloo
2,Algebraic Enumeration,co630,The algebra of formal Laurent series. Multiva...,waterloo
3,Symmetric Functions,co631,"The ring of symmetric functions, standard base...",waterloo
4,Combinatorial Designs,co634,Pairwise orthogonal latin squares. Transversal...,waterloo
...,...,...,...,...
343,Analysis of Longitudinal Data,stat 936,This course covers methods for analyzing data ...,waterloo
344,Statistical Consulting,stat 938,This course will cover some of the basic tools...,waterloo
345,Topics in Probability and Statistics,stat 946,Topics of current interest,waterloo
346,Topics in Biostatistics,stat 947,Topics of current interest,waterloo


In [15]:
course_list.to_csv("waterloo_courses.csv")